In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-msa-common-crawl-qa/resolve/main/common-crawl-qa.json

In [2]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [3]:
with open('common-crawl-qa.json') as fopen:
    data = json.load(fopen)

In [4]:
data_qa = []
for d in data:
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        data_qa.append(
            {
                'context': t,
                'qa': j_a
            }
        )
    except Exception as e:
        pass
    
len(data_qa)

69829

In [5]:
data_qa[0]

{'context': 'Google Map) MAZLAN MOKHTAR JENGKA Jerantut (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) MOHD NAJID JERANTUT Kuantan (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) FAZIRA JALAN TELUK SISEK FAZIRAH ZUKI LORONG CHENGAL LEMPONG NORAZMA KUANTAN AZIZAH KUANTAN NANAD AZRI TAMAN BUKIT RANGIN NURULHANA SUNGAI ISAP ROHANI MUHAMAD JALAN AIR PUTIH (1 - 5) CIK TA JALAN INDUSTRI TANAH PUTIH BARU SAADIAH KAMPUNG BALOK SHARIFAH KUANTAN SITI KHADIJAH KAMPUNG TIRAM SITI NORHAMIZAH TAMAN SERI MAHKOTA AMAN SITI SHARINA KUANTAN ZALIFAH AHMAD BANDAR INDERA MAHKOTA Maran (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) MOHD FADELY BUKIT TAJAU Muadzam Shah (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) HASAZALI PAHANG Pekan (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) FASEEHA KAHAR PEKAN Raub (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) AZNAH TAMAN KEMAJUAN NOR SOLEHAH RAUB NUR SHAFINA RAUB Temerloh (Klik nama untuk WhatsApp, klik lo

In [6]:
indices = {i for i in range(len(data_qa))}

In [7]:
i = 0
for s in data_qa[i]['qa']['soalan']:
    print(s)

{'soalan': 'Siapakah yang berada di Jengka Jerantut?', 'jawapan': 'Mazlan Mokhtar'}
{'soalan': 'Siapakah yang berada di Kuala Kangsar?', 'jawapan': 'Nor Haza atau Rozitah'}
{'soalan': 'Siapakah yang berada di Simpang?', 'jawapan': 'Seri Taman Berkah'}


In [8]:
data_qa[10]['context']

'Camoor Blinds Sdn Bhd About Us Camoor Blinds Sdn Bhd is a window blinds manufacturer and our business covers the entire Malaysia and Singapore. Our main office is located in Kuala Lumpur, Malaysia. Our business model is B2B (business to business). People centric approach has been always our core value. Because in Camoor Blinds , we believe having a great team is our most important assets. Our commitment is to deliver high quality and sleek design as well as locally relevant product to make a real difference to our customers. With this goal in mind, we have never contented and constantly strive for perfection in our products. We believe the best way for us to do this, is through deep understanding of people’s need and desires. We want to bring people meaningful products that make every home and offices and better place to live and work. This sets us apart and make us Camoor Blinds. Home About Us Products Manual Products Outdoor Roller Blinds Roller Blinds Roller Blinds System Smooth Cu

In [13]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data_qa))):
    query = data_qa[i]['context']
    
    for s in data_qa[i]['qa']['soalan']:
        pos = [query]
        
        sampled = random.sample(list(indices - {i}), 100)
        neg = []
        for k in sampled:
            try:
                overlapped = overlap(query, data_qa[k]['context'])
                if overlapped < 0.1:
                    neg.append(data_qa[k]['context'])
                if len(neg) >= 5:
                    break
            except:
                pass
        
        pairs.append({
            'query': s['soalan'],
            'pos': pos,
            'neg': neg
        })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 69829/69829 [56:42<00:00, 20.52it/s]


In [15]:
len(pairs)

209489

In [14]:
with open('common-crawl-qa-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='common-crawl-qa-pair.jsonl',
    path_in_repo='common-crawl-qa-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

common-crawl-qa-pair.jsonl:   0%|          | 0.00/4.19G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/8273e5292cd5bb53f3044ff02300ca0cb358fc55', commit_message='Upload common-crawl-qa-pair.jsonl with huggingface_hub', commit_description='', oid='8273e5292cd5bb53f3044ff02300ca0cb358fc55', pr_url=None, pr_revision=None, pr_num=None)